In [1]:
#modeling


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from statsmodels.tsa.arima.model import ARIMA


In [4]:
df = pd.read_csv("data_no_outliers.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19776 entries, 0 to 19775
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Invoice_id            19776 non-null  int64  
 1   Cust_id               19776 non-null  int64  
 2   Txn_date              19776 non-null  object 
 3   Product_Sub_category  19776 non-null  object 
 4   Product_Category      19776 non-null  object 
 5   Qty                   19776 non-null  int64  
 6   Rate                  19776 non-null  int64  
 7   Tax                   19776 non-null  float64
 8   Total_Amt             19776 non-null  float64
 9   Store_type            19776 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 1.5+ MB


In [7]:
#let's build time based DF's
df['Txn_date'] = pd.to_datetime(df['Txn_date'])
df['Year'] = df['Txn_date'].dt.year
df['Month'] = df['Txn_date'].dt.month
df['Day'] = df['Txn_date'].dt.day
#day of the week and weekends
df['DayOfWeek'] = df['Txn_date'].dt.dayofweek
df['Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

In [9]:
#train and test
X_train, X_test, y_train, y_test = train_test_split(df[['Year', 'Month', 'Day', 'DayOfWeek', 'Weekend']], df['Total_Amt'], test_size=0.2, random_state=0)


In [10]:
model = LinearRegression()

In [12]:
model.fit(X_train, y_train)

LinearRegression()

In [13]:
predictions = model.predict(X_test)

In [17]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('RMSE: ', rmse)

RMSE:  1913.4278258791276


In [18]:
r2 = r2_score(y_test, predictions)
print('R2: ', r2)


R2:  -0.0007239607006916948


In [22]:
df_ts = df.resample('M', on='Txn_date')['Total_Amt'].sum()


In [25]:
model = ARIMA(df_ts, order=(5,1,0))
model_fit = model.fit(disp=0)

plt.figure(figsize=(15,10))
model_fit.plot_predict(dynamic=False)
plt.show()

TypeError: ARIMA.fit() got an unexpected keyword argument 'disp'

In [23]:
model = ARIMA(df_ts['Total_Amt'], order=(5,1,0)) 
model_fit = model.fit(disp=0)


fig, ax = plt.subplots()
ax.plot(df_ts['Total_Amt'], label='original')
ax.plot(model_fit.predict(), label='forecast')
plt.legend()
plt.show()

KeyError: 'Total_Amt'

In [26]:
model_fit = model.fit()


In [27]:
print(model_fit.summary())


                               SARIMAX Results                                
Dep. Variable:              Total_Amt   No. Observations:                   36
Model:                 ARIMA(5, 1, 0)   Log Likelihood                -465.281
Date:                Mon, 24 Jul 2023   AIC                            942.562
Time:                        17:28:49   BIC                            951.894
Sample:                    01-31-2016   HQIC                           945.784
                         - 12-31-2018                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1205      0.045      2.662      0.008       0.032       0.209
ar.L2          0.1478      0.109      1.356      0.175      -0.066       0.361
ar.L3         -0.0371      0.132     -0.282      0.7

In [28]:
plt.figure(figsize=(15,10))
model_fit.plot_predict(dynamic=False)
plt.show()


AttributeError: 'ARIMAResults' object has no attribute 'plot_predict'

<Figure size 1500x1000 with 0 Axes>